In [1]:
import os
import pandas as pd
import numpy as np
import re
import math
from datetime import datetime as dt
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import scipy.stats
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import add_lowess
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

<h1>Reading the data into dataframes</h1>

In [2]:
#reading the csv file in a dataframe
conviva = pd.read_csv('C:\\Users\\AADITYA\\Documents\\project_data\\sample_dataset\\conviva_sampledata.csv')
tv_cms = pd.read_csv('C:\\Users\\AADITYA\\Documents\\project_data\\sample_dataset\\tv_cms_06092019_without_genre.csv')
movie_cms = pd.read_csv('C:\\Users\\AADITYA\\Documents\\project_data\\sample_dataset\\movie_cms_06092019_without_genre_businesstype.csv')

In [3]:
conviva_notna = conviva.dropna()

In [4]:
conviva_notna.shape

(2313090, 24)

In [5]:
conviva_notna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2313090 entries, 0 to 2352710
Data columns (total 24 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   Unnamed: 0                   int64 
 1   viewerid                     object
 2   asset                        object
 3   deviceos                     object
 4   country                      object
 5   state                        object
 6   city                         object
 7   asn                          int64 
 8   isp                          object
 9   starttime                    int64 
 10  startuptime                  int64 
 11  playingtime                  int64 
 12  bufferingtime                int64 
 13  interrupts                   int64 
 14  averagebitrate               int64 
 15  startuperror                 int64 
 16  ipaddress                    object
 17  cdn                          object
 18  sessiontag_c3_player_name    object
 19  sessiontag_c3_client_

In [6]:
conviva_notna.head()

,Unnamed: 0,viewerid,asset,deviceos,country,state,city,asn,isp,starttime,...,averagebitrate,startuperror,ipaddress,cdn,sessiontag_c3_player_name,sessiontag_c3_client_brand,sessiontag_connectiontype,sessiontag_c3_client_hwtype,sessiontag_accesstype,churn
0,0,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-27132] Kedarnath,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559557555,...,0,0,122.168.85.95,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
1,1,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35166] Simmba,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559557880,...,0,0,122.168.85.95,AMAZON,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
2,2,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35021] Nirahua Hindustani 3,Android,india,madhya pradesh,indore,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559448731,...,0,0,122.170.203.36,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
3,3,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-1-215897] Skyfire - Episode 8 - The Monster,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,1561486982,...,0,0,103.219.230.183,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1
4,4,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-0-6956] Great Grand Masti,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,1561834149,...,0,0,103.219.229.92,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1


<h1>Finding out number of assets each viewer has watched</h1>

In [7]:
model_data = conviva_notna.groupby(['viewerid','churn'])['asset'].count().reset_index()

In [8]:
model_data.head()

,viewerid,churn,asset
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6
4,0000f172-8523-4a97-809f-846475b0bd79,0,5


In [9]:
model_data.rename(columns = {'asset':'platform_visits'}, inplace = True)

In [10]:
model_data

,viewerid,churn,platform_visits
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6
4,0000f172-8523-4a97-809f-846475b0bd79,0,5
...,...,...,...
174175,fffec46b-3722-4a70-8dc8-990e29309e9a,0,16
174176,ffff6f1d-490b-4895-92e4-16b2e3bc3d9b,0,49
174177,ffffa05b-a7b5-486d-9501-80911d6f9cdd,0,10
174178,ffffce20-c435-4695-82c7-fe24e23facd9,0,12


In [11]:
model_data.shape

(174180, 3)

<h1>Time related and interrupt calculations</h1>

In [12]:
time_data = conviva_notna.groupby('viewerid').agg(
{
    'playingtime':'mean',
    'startuptime':'mean',
    'bufferingtime':'mean',
    'interrupts':'mean'
}
)

In [13]:
time_data = time_data.reset_index()

In [14]:
time_data.head()

,viewerid,playingtime,startuptime,bufferingtime,interrupts
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,0.000000e+00,-1.000000,0.000000,0.0
1,00008f8d-cb75-4048-9f4c-21995eba4215,1.521220e+06,6721.000000,41198.360000,9.4
2,0000b45b-756e-4cc5-8230-515ee195e31f,3.476752e+06,3834.666667,2514.333333,4.0
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,4.223742e+05,2205.666667,20434.833333,40.5
4,0000f172-8523-4a97-809f-846475b0bd79,4.252400e+03,2284.400000,5446.000000,0.2


In [15]:
#joining this data with the model data

In [16]:
model_data = pd.merge(model_data, time_data, how = 'inner', on = 'viewerid')

In [17]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2


In [18]:
#

In [19]:
model_data['buffer_per_playtime'] = model_data['bufferingtime'] / model_data['playingtime']

In [20]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689


In [21]:
model_data['interrupts_to_playtime'] = model_data['interrupts'] / model_data['playingtime']

In [22]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN,NaN
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082,0.000006
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723,0.000001
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381,0.000096
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689,0.000047


In [23]:
#

In [24]:
model_data['interrupts_per_visit'] = model_data['interrupts'] / model_data['platform_visits']

In [25]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime,interrupts_per_visit
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN,NaN,0.000000
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082,0.000006,0.376000
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723,0.000001,1.333333
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381,0.000096,6.750000
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689,0.000047,0.040000


In [26]:
#

In [27]:
model_data['waste_time'] = model_data['bufferingtime'] + model_data['startuptime']

In [28]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime,interrupts_per_visit,waste_time
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN,NaN,0.000000,-1.00
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082,0.000006,0.376000,47919.36
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723,0.000001,1.333333,6349.00
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381,0.000096,6.750000,22640.50
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689,0.000047,0.040000,7730.40


In [29]:
#

In [30]:
model_data['waste_time_per_visit'] = model_data['waste_time'] / model_data['platform_visits']

In [31]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime,interrupts_per_visit,waste_time,waste_time_per_visit
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN,NaN,0.000000,-1.00,-0.500000
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082,0.000006,0.376000,47919.36,1916.774400
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723,0.000001,1.333333,6349.00,2116.333333
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381,0.000096,6.750000,22640.50,3773.416667
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689,0.000047,0.040000,7730.40,1546.080000


In [32]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174180 entries, 0 to 174179
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   viewerid                174180 non-null  object 
 1   churn                   174180 non-null  int64  
 2   platform_visits         174180 non-null  int64  
 3   playingtime             174180 non-null  float64
 4   startuptime             174180 non-null  float64
 5   bufferingtime           174180 non-null  float64
 6   interrupts              174180 non-null  float64
 7   buffer_per_playtime     168982 non-null  float64
 8   interrupts_to_playtime  168982 non-null  float64
 9   interrupts_per_visit    174180 non-null  float64
 10  waste_time              174180 non-null  float64
 11  waste_time_per_visit    174180 non-null  float64
dtypes: float64(9), int64(2), object(1)
memory usage: 17.3+ MB


<h1>Finding the unique number of assets each viewer has watched and calculations based on it</h1>

In [33]:
unique_assets = conviva_notna.groupby('viewerid')['asset'].nunique()

In [34]:
unique_assets = unique_assets.reset_index()

In [35]:
unique_assets

,viewerid,asset
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,2
1,00008f8d-cb75-4048-9f4c-21995eba4215,16
2,0000b45b-756e-4cc5-8230-515ee195e31f,3
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,5
4,0000f172-8523-4a97-809f-846475b0bd79,5
...,...,...
174175,fffec46b-3722-4a70-8dc8-990e29309e9a,12
174176,ffff6f1d-490b-4895-92e4-16b2e3bc3d9b,39
174177,ffffa05b-a7b5-486d-9501-80911d6f9cdd,5
174178,ffffce20-c435-4695-82c7-fe24e23facd9,6


In [36]:
model_data = pd.merge(model_data, unique_assets, how = 'inner', on = 'viewerid')

In [37]:
model_data

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime,interrupts_per_visit,waste_time,waste_time_per_visit,asset
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.000000,NaN,NaN,0.000000,-1.000000,-0.500000,2
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.400000,0.027082,0.000006,0.376000,47919.360000,1916.774400,16
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.000000,0.000723,0.000001,1.333333,6349.000000,2116.333333,3
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.500000,0.048381,0.000096,6.750000,22640.500000,3773.416667,5
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.200000,1.280689,0.000047,0.040000,7730.400000,1546.080000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174175,fffec46b-3722-4a70-8dc8-990e29309e9a,0,16,5.017922e+05,2810.750000,6650.812500,3.750000,0.013254,0.000007,0.234375,9461.562500,591.347656,12
174176,ffff6f1d-490b-4895-92e4-16b2e3bc3d9b,0,49,2.613446e+05,4711.183673,50081.734694,14.714286,0.191631,0.000056,0.300292,54792.918367,1118.222824,39
174177,ffffa05b-a7b5-486d-9501-80911d6f9cdd,0,10,1.153465e+06,1876.500000,95397.300000,10.900000,0.082705,0.000009,1.090000,97273.800000,9727.380000,5
174178,ffffce20-c435-4695-82c7-fe24e23facd9,0,12,1.584172e+06,3417.833333,10999.000000,12.416667,0.006943,0.000008,1.034722,14416.833333,1201.402778,6


In [38]:
#

In [39]:
model_data['asset_to_visit_ratio'] = model_data['asset'] / model_data['platform_visits']

In [40]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime,interrupts_per_visit,waste_time,waste_time_per_visit,asset,asset_to_visit_ratio
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN,NaN,0.000000,-1.00,-0.500000,2,1.000000
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082,0.000006,0.376000,47919.36,1916.774400,16,0.640000
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723,0.000001,1.333333,6349.00,2116.333333,3,1.000000
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381,0.000096,6.750000,22640.50,3773.416667,5,0.833333
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689,0.000047,0.040000,7730.40,1546.080000,5,1.000000


In [41]:
#

In [42]:
model_data['interrupts_per_asset'] = model_data['interrupts'] / model_data['asset']

In [43]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime,interrupts_per_visit,waste_time,waste_time_per_visit,asset,asset_to_visit_ratio,interrupts_per_asset
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN,NaN,0.000000,-1.00,-0.500000,2,1.000000,0.000000
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082,0.000006,0.376000,47919.36,1916.774400,16,0.640000,0.587500
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723,0.000001,1.333333,6349.00,2116.333333,3,1.000000,1.333333
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381,0.000096,6.750000,22640.50,3773.416667,5,0.833333,8.100000
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689,0.000047,0.040000,7730.40,1546.080000,5,1.000000,0.040000


In [44]:
#

In [45]:
model_data['waste_time_per_asset'] = model_data['waste_time'] / model_data['asset']

In [46]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime,interrupts_per_visit,waste_time,waste_time_per_visit,asset,asset_to_visit_ratio,interrupts_per_asset,waste_time_per_asset
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN,NaN,0.000000,-1.00,-0.500000,2,1.000000,0.000000,-0.500000
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082,0.000006,0.376000,47919.36,1916.774400,16,0.640000,0.587500,2994.960000
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723,0.000001,1.333333,6349.00,2116.333333,3,1.000000,1.333333,2116.333333
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381,0.000096,6.750000,22640.50,3773.416667,5,0.833333,8.100000,4528.100000
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689,0.000047,0.040000,7730.40,1546.080000,5,1.000000,0.040000,1546.080000


In [47]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174180 entries, 0 to 174179
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   viewerid                174180 non-null  object 
 1   churn                   174180 non-null  int64  
 2   platform_visits         174180 non-null  int64  
 3   playingtime             174180 non-null  float64
 4   startuptime             174180 non-null  float64
 5   bufferingtime           174180 non-null  float64
 6   interrupts              174180 non-null  float64
 7   buffer_per_playtime     168982 non-null  float64
 8   interrupts_to_playtime  168982 non-null  float64
 9   interrupts_per_visit    174180 non-null  float64
 10  waste_time              174180 non-null  float64
 11  waste_time_per_visit    174180 non-null  float64
 12  asset                   174180 non-null  int64  
 13  asset_to_visit_ratio    174180 non-null  float64
 14  interrupts_per_asset

<h1>Stationary vs mobile usage</h1>

In [48]:
stationary_or_mobile_users = conviva_notna[['viewerid','sessiontag_connectiontype','sessiontag_c3_client_hwtype']]

In [49]:
s = stationary_or_mobile_users['sessiontag_connectiontype']

stationary_or_mobile_users['mobile_connection'] = s.map(lambda x:1 if x=='Data'
                                        else 2)

In [50]:
t = stationary_or_mobile_users['sessiontag_c3_client_hwtype']

stationary_or_mobile_users['mobile_device'] = s.map(lambda x:1 if x=='Mobile+Phone'
                                        else 2)

In [51]:
stationary_or_mobile_users.head()

,viewerid,sessiontag_connectiontype,sessiontag_c3_client_hwtype,mobile_connection,mobile_device
0,0171fdb9-d8db-4e00-a9d8-c1619811ac66,Data,TV,1,2
1,0171fdb9-d8db-4e00-a9d8-c1619811ac66,Data,TV,1,2
2,0171fdb9-d8db-4e00-a9d8-c1619811ac66,Data,TV,1,2
3,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,WiFi,Mobile+Phone,2,2
4,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,WiFi,Mobile+Phone,2,2


In [52]:
stationary_or_mobile_users.head()

,viewerid,sessiontag_connectiontype,sessiontag_c3_client_hwtype,mobile_connection,mobile_device
0,0171fdb9-d8db-4e00-a9d8-c1619811ac66,Data,TV,1,2
1,0171fdb9-d8db-4e00-a9d8-c1619811ac66,Data,TV,1,2
2,0171fdb9-d8db-4e00-a9d8-c1619811ac66,Data,TV,1,2
3,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,WiFi,Mobile+Phone,2,2
4,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,WiFi,Mobile+Phone,2,2


<h1>weekends or weekdays</h1>

In [53]:
#converting the UNIX timestamps to datetime
conviva_notna['starttime'] = pd.to_datetime(conviva_notna['starttime'],unit='s')
#adding the calculated weekday column to the conviva table
conviva_notna['weekday'] = conviva_notna['starttime'].dt.weekday

In [54]:
conviva_weekday_end_calculation = conviva_notna[['viewerid','starttime','weekday']]

In [55]:
s = conviva_weekday_end_calculation['weekday']

conviva_weekday_end_calculation['weekday_count'] = s.map(lambda x:1 if x < 5
                                        else 2)

In [56]:
weekdays_watched = conviva_weekday_end_calculation.groupby('viewerid')['weekday_count'].apply(lambda x: (x==1).sum()).reset_index(name='watched_on_weekdays')

In [57]:
weekends_watched = conviva_weekday_end_calculation.groupby('viewerid')['weekday_count'].apply(lambda x: (x==2).sum()).reset_index(name='watched_on_weekends')

In [58]:
model_data = pd.merge(model_data, weekdays_watched, how='inner', on='viewerid')

In [59]:
model_data = pd.merge(model_data, weekends_watched, how='inner', on='viewerid')

In [60]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime,interrupts_per_visit,waste_time,waste_time_per_visit,asset,asset_to_visit_ratio,interrupts_per_asset,waste_time_per_asset,watched_on_weekdays,watched_on_weekends
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN,NaN,0.000000,-1.00,-0.500000,2,1.000000,0.000000,-0.500000,1,1
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082,0.000006,0.376000,47919.36,1916.774400,16,0.640000,0.587500,2994.960000,15,10
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723,0.000001,1.333333,6349.00,2116.333333,3,1.000000,1.333333,2116.333333,2,1
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381,0.000096,6.750000,22640.50,3773.416667,5,0.833333,8.100000,4528.100000,1,5
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689,0.000047,0.040000,7730.40,1546.080000,5,1.000000,0.040000,1546.080000,0,5


<h1>
    Calculating how many times has each viewer watched content late at night, or in the morning, afternoon, evening or night
    respectively
</h1>

In [61]:
s = conviva_notna['starttime']

conviva_notna['time_of_day'] = s.map(lambda x:1 if x.hour in range(4,12) 
                                        else(2 if x.hour in range(12,17)
                                            else(3 if x.hour in range(18,21)
                                                else(4 if x.hour in range(21,23)
                                                    else 0
                                                    )
                                                )
                                            )
                                    )

In [62]:
conviva_time_of_day = conviva_notna[['viewerid','time_of_day']]

In [63]:
conviva_late_night_watched = conviva_time_of_day.groupby('viewerid')['time_of_day'].apply(lambda x: (x==0).sum()).reset_index(name='late_night_watched')
conviva_morning_watched = conviva_time_of_day.groupby('viewerid')['time_of_day'].apply(lambda x: (x==1).sum()).reset_index(name='morning_watched')
conviva_afternoon_watched = conviva_time_of_day.groupby('viewerid')['time_of_day'].apply(lambda x: (x==2).sum()).reset_index(name='afternoon_watched')
conviva_evening_watched = conviva_time_of_day.groupby('viewerid')['time_of_day'].apply(lambda x: (x==3).sum()).reset_index(name='evening_watched')
conviva_night_watched = conviva_time_of_day.groupby('viewerid')['time_of_day'].apply(lambda x: (x==4).sum()).reset_index(name='night_watched')

In [64]:
model_data = pd.merge(model_data, conviva_late_night_watched, how='inner', on='viewerid')

In [65]:
model_data = pd.merge(model_data, conviva_morning_watched, how='inner', on='viewerid')

In [66]:
model_data = pd.merge(model_data, conviva_afternoon_watched, how='inner', on='viewerid')

In [67]:
model_data = pd.merge(model_data, conviva_evening_watched, how='inner', on='viewerid')

In [68]:
model_data = pd.merge(model_data, conviva_night_watched, how='inner', on='viewerid')

In [69]:
model_data.head()

,viewerid,churn,platform_visits,playingtime,startuptime,bufferingtime,interrupts,buffer_per_playtime,interrupts_to_playtime,interrupts_per_visit,...,asset_to_visit_ratio,interrupts_per_asset,waste_time_per_asset,watched_on_weekdays,watched_on_weekends,late_night_watched,morning_watched,afternoon_watched,evening_watched,night_watched
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,2,0.000000e+00,-1.000000,0.000000,0.0,NaN,NaN,0.000000,...,1.000000,0.000000,-0.500000,1,1,1,1,0,0,0
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,25,1.521220e+06,6721.000000,41198.360000,9.4,0.027082,0.000006,0.376000,...,0.640000,0.587500,2994.960000,15,10,8,11,6,0,0
2,0000b45b-756e-4cc5-8230-515ee195e31f,0,3,3.476752e+06,3834.666667,2514.333333,4.0,0.000723,0.000001,1.333333,...,1.000000,1.333333,2116.333333,2,1,0,0,1,2,0
3,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,6,4.223742e+05,2205.666667,20434.833333,40.5,0.048381,0.000096,6.750000,...,0.833333,8.100000,4528.100000,1,5,1,4,1,0,0
4,0000f172-8523-4a97-809f-846475b0bd79,0,5,4.252400e+03,2284.400000,5446.000000,0.2,1.280689,0.000047,0.040000,...,1.000000,0.040000,1546.080000,0,5,0,4,1,0,0


<h1>Logistic Regression #1</h1>

In [70]:
model_data = model_data.replace([np.inf, -np.inf], np.nan)
model_data = model_data.fillna(0)

In [71]:
x = model_data
x = x.drop(['viewerid','churn'], axis = 1)
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",math.ceil(accuracy_score(ytest,ypred) * 100),"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 65 %
The confusion matrix is: 


array([[22299,   192],
       [12032,   313]], dtype=int64)

In [72]:
#top 3 factors

In [73]:
conviva_notna['asset'].value_counts().nlargest(10)

[0-0-39320] The Accidental Prime Minister              41470
[0-1-226358] Kaafir - Episode 1 - The First Meeting    41062
[0-0-33204] Uri: The Surgical Strike                   40665
[0-0-35166] Simmba                                     30314
[0-0-27132] Kedarnath                                  27031
[0-0-59341] Jersey                                     20526
[0-0-47046] Why Cheat India                            18150
[0-1-215883] Skyfire - Skyfire - Episode 1 - Pralay    15326
[0-9-zeemarathi] Zee Marathi                           14002
[0-0-movie_1424717826] Veere Di Wedding                13964
Name: asset, dtype: int64

In [74]:
conviva_speed = conviva.groupby('isp').agg({
    'bufferingtime':'mean',
    'averagebitrate':'mean'
}).reset_index()

In [75]:
conviva_notna['isp'].value_counts().nlargest(5)

ASN: RELIANCEJIO-IN Reliance Jio Infocomm Limited, IN                                                          445384
Bharti Airtel                                                                                                  369300
ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd., Telemedia Services, IN                                          192215
Vodafone                                                                                                       181268
ASN: CABLELITE-AS-AP Atria Convergence Technologies Pvt. Ltd. Broadband Internet Service Provider INDIA, IN    104322
Name: isp, dtype: int64

In [76]:
conviva_notna['sessiontag_c3_client_brand'].value_counts().nlargest(3)

Xiaomi     513526
Samsung    348993
Amazon     234167
Name: sessiontag_c3_client_brand, dtype: int64

In [77]:
conviva_device_speed = conviva_notna[['sessiontag_c3_client_hwtype','averagebitrate']]

In [78]:
conviva_device_speed['sessiontag_c3_client_hwtype'].value_counts()

Mobile+Phone             1651857
Set+Top+Box               258452
TV                        211268
Desktop                   160704
Tablet                     30726
Media+Player                  63
Games+Console                 14
Single-board+Computer          3
Payment+Terminal               3
Name: sessiontag_c3_client_hwtype, dtype: int64

In [79]:
conviva_hw_bitrate = conviva_notna.groupby('sessiontag_c3_client_hwtype')['averagebitrate'].mean().reset_index()

In [80]:
conviva_hw_bitrate.sort_values('averagebitrate', ascending = False)

,sessiontag_c3_client_hwtype,averagebitrate
0,Desktop,1237.005725
8,Tablet,808.756558
3,Mobile+Phone,678.378535
2,Media+Player,276.539683
5,Set+Top+Box,120.388734
1,Games+Console,61.571429
7,TV,37.942045
4,Payment+Terminal,0.000000
6,Single-board+Computer,0.000000


In [81]:
conviva_notna['churn'].value_counts()

0    1916934
1     396156
Name: churn, dtype: int64

In [82]:
notchurn = conviva_notna['churn'].value_counts().values[0] / len(conviva_notna)

In [83]:
notchurn * 100

82.87329935281377

In [84]:
churn = conviva_notna['churn'].value_counts().values[1] / len(conviva_notna)

In [85]:
churn * 100

17.126700647186233

In [86]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174180 entries, 0 to 174179
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   viewerid                174180 non-null  object 
 1   churn                   174180 non-null  int64  
 2   platform_visits         174180 non-null  int64  
 3   playingtime             174180 non-null  float64
 4   startuptime             174180 non-null  float64
 5   bufferingtime           174180 non-null  float64
 6   interrupts              174180 non-null  float64
 7   buffer_per_playtime     174180 non-null  float64
 8   interrupts_to_playtime  174180 non-null  float64
 9   interrupts_per_visit    174180 non-null  float64
 10  waste_time              174180 non-null  float64
 11  waste_time_per_visit    174180 non-null  float64
 12  asset                   174180 non-null  int64  
 13  asset_to_visit_ratio    174180 non-null  float64
 14  interrupts_per_asset

In [87]:
#model 1
x = model_data[['platform_visits']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 64.56252152945228 %
The confusion matrix is: 


array([[22491,     0],
       [12345,     0]], dtype=int64)

In [88]:
#model 2
x = model_data[['platform_visits',
                'playingtime']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 64.56252152945228 %
The confusion matrix is: 


array([[22491,     0],
       [12345,     0]], dtype=int64)

In [89]:
#model 3
x = model_data[['platform_visits',
               'playingtime',
               'startuptime']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 64.94431048340797 %
The confusion matrix is: 


array([[22033,   458],
       [11754,   591]], dtype=int64)

In [90]:
#model 4
x = model_data[['platform_visits',
               'playingtime',
               'startuptime',
               'bufferingtime']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 64.93856929613044 %
The confusion matrix is: 


array([[22029,   462],
       [11752,   593]], dtype=int64)

In [91]:
#model 5
x = model_data[['platform_visits',
               'playingtime',
               'startuptime',
               'bufferingtime',
               'interrupts']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 64.93856929613044 %
The confusion matrix is: 


array([[22029,   462],
       [11752,   593]], dtype=int64)

In [92]:
#model 6
x = model_data[['platform_visits',
               'playingtime',
               'startuptime',
               'bufferingtime',
               'interrupts',
               'buffer_per_playtime']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 64.93856929613044 %
The confusion matrix is: 


array([[22029,   462],
       [11752,   593]], dtype=int64)

In [93]:
#model 7
x = model_data[['platform_visits',
               'playingtime',
               'startuptime',
               'bufferingtime',
               'interrupts',
               'buffer_per_playtime',
               'asset_to_visit_ratio',
               'asset']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 64.93856929613044 %
The confusion matrix is: 


array([[22029,   462],
       [11752,   593]], dtype=int64)

In [94]:
#model 8
x = model_data[['platform_visits',
               'playingtime',
               'startuptime',
               'bufferingtime',
               'interrupts',
               'buffer_per_playtime',
               'asset_to_visit_ratio',
               'asset',
               'waste_time_per_visit']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 64.87541623607763 %
The confusion matrix is: 


array([[22310,   181],
       [12055,   290]], dtype=int64)

In [95]:
#model 9
x = model_data[['platform_visits',
               'playingtime',
               'startuptime',
               'bufferingtime',
               'interrupts',
               'buffer_per_playtime',
               'asset_to_visit_ratio',
               'asset',
               'waste_time_per_visit',
               'watched_on_weekdays',
               'watched_on_weekends']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 65.35767596739007 %
The confusion matrix is: 


array([[21796,   695],
       [11373,   972]], dtype=int64)

In [96]:
#model 10
x = model_data[['platform_visits',
               'playingtime',
               'startuptime',
               'bufferingtime',
               'interrupts',
               'buffer_per_playtime',
               'asset_to_visit_ratio',
               'asset',
               'waste_time_per_visit',
               'watched_on_weekdays',
               'watched_on_weekends',
               'waste_time']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 65.35767596739007 %
The confusion matrix is: 


array([[21796,   695],
       [11373,   972]], dtype=int64)

In [97]:
#model 10
x = model_data[['platform_visits',
               'playingtime',
               'startuptime',
               'bufferingtime',
               'interrupts',
               'buffer_per_playtime',
               'asset_to_visit_ratio',
               'asset',
               'waste_time_per_visit',
               'watched_on_weekdays',
               'watched_on_weekends',
               'buffer_per_playtime',
               'interrupts_to_playtime']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 65.35767596739007 %
The confusion matrix is: 


array([[21796,   695],
       [11373,   972]], dtype=int64)

In [98]:
#model 11
x = model_data[['platform_visits',
               'playingtime',
               'startuptime',
               'bufferingtime',
               'interrupts',
               'buffer_per_playtime',
               'asset_to_visit_ratio',
               'asset',
               'waste_time_per_visit',
               'watched_on_weekdays',
               'watched_on_weekends',
               'buffer_per_playtime',
               'interrupts_to_playtime',
               'late_night_watched',
               'morning_watched']]
y = model_data['churn']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)   
lg = LogisticRegression(solver='lbfgs')
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)
print("The accuracy of model #6 is",accuracy_score(ytest,ypred) * 100,"%")
print("The confusion matrix is: ")
confusion_matrix(ytest,ypred)

The accuracy of model #6 is 65.35767596739007 %
The confusion matrix is: 


array([[21796,   695],
       [11373,   972]], dtype=int64)

In [99]:
print("The classification report is: ")
classification_report(ytest,ypred)

The classification report is: 


'              precision    recall  f1-score   support\n\n           0       0.66      0.97      0.78     22491\n           1       0.58      0.08      0.14     12345\n\n    accuracy                           0.65     34836\n   macro avg       0.62      0.52      0.46     34836\nweighted avg       0.63      0.65      0.55     34836\n'

In [103]:
#model 5
model = LogisticRegression(solver='lbfgs', random_state=0)
x = model_data[[
               'playingtime',
               'startuptime',
               'bufferingtime',
               'interrupts',
               'asset',
               ]]
y = model_data['churn']
model.fit(x, y)
print("The accuracy of the model is ", model.score(x, y) * 100, "%")
print("The confusion matrix is as follows: \n")
print(confusion_matrix(y, model.predict(x)))
print("The classification report is as follows: \n")
print(classification_report(y, model.predict(x)))

The accuracy of the model is  64.87943506717188 %
The confusion matrix is as follows: 

[[109896   2177]
 [ 58996   3111]]
The classification report is as follows: 

              precision    recall  f1-score   support

           0       0.65      0.98      0.78    112073
           1       0.59      0.05      0.09     62107

    accuracy                           0.65    174180
   macro avg       0.62      0.52      0.44    174180
weighted avg       0.63      0.65      0.54    174180

